In [ ]:
with cte as
(
select *,max(score) over(partition by exam_id) as max_score,
min(score) over(partition by exam_id) as min_score
from Exam
),
cte2 as 
(
select distinct student_id from cte where score =max_score or score=min_score
),
cte3 as (
select distinct student_id from cte 
where (score <> max_score or score <> min_score) and student_id not in(select student_id from cte2)
)

select * from student where student_id in (select student_id from cte3)
order by student_id;

In [ ]:
with cte as (
select 'failed' as period_state,fail_date as event_date 
from Failed where strftime('%Y',fail_date)='2019'
union all
select  'succeeded' as period_state,success_date as event_date 
from Succeeded where strftime('%Y',success_date)='2019'), cte1 as (
select *,lag(period_state,1,period_state) over(order by event_date) as prev_state 
from cte),cte2 as (
    select *, 
    sum(case when period_state=prev_state then 0 else 1 end) over(order by event_date) as group_ 
    from cte1
)
select period_state,min(event_date)as start_date,max(event_date) as end_date from cte2
group by period_state,group_
order by start_date;

In [ ]:
with recursive cte as (
select employee_id from Employees
where manager_id=1 and employee_id<>1
union
select e.employee_id from Employees as e
inner join cte as c
on e.manager_id=c.employee_id
)
select * from cte
order by employee_id;

In [ ]:
with recursive cte as(
select task_id,subtasks_count,1 as subtask_id
from Tasks
union all
select task_id,subtasks_count,subtask_id+1 as subtask_id
from cte where subtask_id<subtasks_count)

select task_id,subtask_id
from cte
where (task_id,subtask_id) not in (select task_id,subtask_id from Executed)
order by task_id;